In [1]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import csv
import numpy as np

sia =SentimentIntensityAnalyzer()
def get_sentiment_of(sentence_in):
    try:
        scrs=sia.polarity_scores(sentence_in)
        the_important_one=scrs['compound']
    except:
        the_important_one=0
    return the_important_one

df = pd.read_csv('yelp_reviews_chicago_labeled.csv')
df2 = df.drop(['reviewID', 'restaurantID', 'reviewerID', 'date', 'name', 'location', 'yelpJoinDate', 'mnr', 'rl', 'rd', 'Maximum Content Similarity'], axis = 1)
df2 = df2.drop(['friendCount', 'reviewCount', 'firstCount', 'usefulCount', 'complimentCount', 'tipCount', 'fanCount', 'restaurantRating'], axis = 1)
f = df2['reviewContent']
sent = list()
for rows in f:
    #print(rows)
    sent.append(get_sentiment_of(rows))
print(len(sent))

df2['Sentiment'] = sent
df_final = df2.drop(['reviewContent'], axis = 1)
fields_with_nan_length = df_final.isna().sum().where(lambda x : x > 0).dropna()
df_final = df_final.iloc[:-1, :]
authentic_reviews_df = df_final[df_final['flagged'] == 'N']
fake_reviews_df = df_final[df_final['flagged'] == 'Y']
authentic_reviews_df = authentic_reviews_df.sample(n = 500)
df_final = pd.concat([authentic_reviews_df , fake_reviews_df])
df_final['flagged'] = [0 if x == 'N' else 1 for x in df_final['flagged']]
flagged = df_final['flagged']
df_final = df_final.drop(['flagged'], axis=1)
df_final = pd.concat([df_final,flagged], axis=1)
X = df_final.iloc[:, :-1]
y = flagged
X.head()



3626


,rating,reviewUsefulCount,coolCount,funnyCount,Sentiment
1124,5,0,2.0,2.0,0.9387
725,5,0,4.0,7.0,0.9260
223,4,5,29.0,24.0,0.9805
2501,5,3,838.0,668.0,0.8957
125,5,17,1476.0,844.0,0.9979


In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [4]:
clf = RandomForestClassifier(n_estimators = 50,
                                criterion = 'entropy',
                                max_depth = 5,
                                bootstrap = True)

scores = cross_val_score(clf, X, y, cv = 10)
print(scores)

from sklearn.model_selection import cross_validate

print("Overal Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.80612245 0.84693878 0.81632653 0.80612245 0.84536082 0.86597938
 0.78350515 0.84536082 0.90721649 0.80412371]
Overal Accuracy: 0.83 (+/- 0.07)


In [5]:
train_copy = df_final.drop(['flagged'], axis=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
feat_labels = train_copy.columns[0:]
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%2d %-*s %f" %(f+1,30,feat_labels[indices[f]],importances[indices[f]]))

 1 funnyCount                     0.350138
 2 reviewUsefulCount              0.303711
 3 coolCount                      0.248874
 4 Sentiment                      0.069469
 5 rating                         0.027808


In [6]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.91      0.76      0.83       146
     class 1       0.80      0.93      0.86       147

   micro avg       0.84      0.84      0.84       293
   macro avg       0.85      0.84      0.84       293
weighted avg       0.85      0.84      0.84       293

